In [1]:
import lionagi as li
from pydantic import Field
import asyncio

This notebook demonstrates how to create a custom worker, `Coder`, that can handle various coding tasks. The `Coder` worker follows a structured workflow, breaking down tasks into planning, writing, and reviewing code. 

In [2]:
"""
- plan code
- write code
- review code
"""

'\n- plan code\n- write code\n- review code\n'

# Define the Coder Form Template

To handle specific tasks effectively, we need to define a form template that will be used to structure and store the data required for each task. In this example, we'll create a CoderForm class that inherits from `li.Form`. This form will include fields for the context, plan, code, and review.

In [3]:
class CoderForm(li.Form):
    """
    A form template for developing software project
    """

    template_name: str = "coder template"

    context: str | None= Field(
        None,
        title="Context",
        description="The context for the current issue to achieve",
    )

    plan: str | None = Field(
        None,
        title="Plan",
        description=" breaking down the issue to multi-step plan",
    )

    code: str | None = Field(
        None,
        title="Code",
        description="codes generated by workflow",
    )

    review: str | None = Field(
        None,
        title="review",
        description="comments on generated code"
    )


In [4]:
GUIDANCE_RESPONSE = """
    Guidance from super intelligent code bot:
    {guidance_response}
    Please generate Python functions that satisfies the prompt and follows the provided guidance, while adhering to these coding standards:
    - Use descriptive and meaningful names for variables, functions, and classes.
    - Follow the naming conventions: lowercase with underscores for functions and variables, CamelCase for classes.
    - Keep functions small and focused, doing one thing well.
    - Use 4 spaces for indentation, and avoid mixing spaces and tabs.
    - Limit line length to 79 characters for better readability.
    - Use docstrings to document functions, classes, and modules, describing their purpose, parameters, and return values.
    - Use comments sparingly, and prefer descriptive names and clear code structure over comments.
    - Handle exceptions appropriately and raise exceptions with clear error messages.
    - Use blank lines to separate logical sections of code, but avoid excessive blank lines.
    - Import modules in a specific order: standard library, third-party, and local imports, separated by blank lines.
    - Use consistent quotes (single or double) for strings throughout the codebase.
    - Follow the PEP 8 style guide for more detailed coding standards and best practices.
"""

PLAN_PROMPT = "Please design coding instructions for the following prompt and provide guidance for the coder to follow."
WRITE_PROMPT = "Please write a Python function that satisfies the prompt and follows the provided guidance."
REVIEW_PROMPT = "Please review the following code and remove any unnecessary markdown or descriptions:\n\n{code}\n"

CODER_PROMPTS = {
    "system": GUIDANCE_RESPONSE,
    "plan_code": PLAN_PROMPT,
    "write_code": WRITE_PROMPT,
    "review_code": REVIEW_PROMPT,
}

# Define the Coder Worker

In [5]:
class Coder(li.Worker):
    name = "coder"
    form_template = CoderForm

    def __init__(self, session=None, system=None, session_kwargs=None):
        """
        Initializes the Coder instance. It sets up the session and tasks dictionary. 
        If no session is provided, it creates a new session using the provided system and session_kwargs.
        """
        super().__init__()
        self.session = session or self._create_session(system, session_kwargs)
        self.tasks = {}
    
    def _create_session(self, system=None, session_kwargs=None):
        """
        Creates a new session with the given system and session_kwargs. 
        This session is used to manage branches and tasks.
        """
        session_kwargs = session_kwargs or {}
        session = li.Session(system=system, **session_kwargs)
        return session
    
    def create_new_issue(self, issue):
        """
        Creates a new issue for the Coder to work on. 
        It initializes a new form based on the CoderForm template and associates it with the issue. 
        Each issue corresponds to a unique branch in the session.
        """
        if self.forms.get(self.session.branches[0].ln_id):
            new_branch = self.session.new_branch()
            self.forms[new_branch.ln_id] = self.form_template(context = issue)
        else:
            new_branch = self.session.default_branch
            self.forms[new_branch.ln_id] = self.form_template(context = issue)
            self.default_form = self.forms[new_branch.ln_id]

    async def _update_form(self, work):
        """
        Updates the form with the results of the completed work. 
        It iterates through the requested fields at each step and updates the corresponding values in the form.
        """
        if work.status == "COMPLETED":
            for field in work.result.requested_fields:
                value = getattr(work.result, field)
                setattr(work.result.origin, field, value)

    """
    @li.work:
        assignment: The expected input and output fields at this step, e.g., "input1, input2 -> output".
                    Note:
                    - All input fields should be filled before the step execution.
                    - Check all available forms in the worker at self.forms.
        form_param_key: The function parameter that is used as the key to map the corresponding form in self.forms.
                        This feature helps you create a form based on the assignment and matched form internally.
                        For example, in this coder example, we use branch ln_id for form mapping,
                        as each branch solves one coding issue, and each issue corresponds to one form.
                        Note:
                        - The key can be any Python-supported dict key types or LionIDable objects.
        guidance: The task of the work.
                Note: If guidance is not provided explicitly, the docstring of the function will be used as the guidance if available.
        capacity: The capacity of the work queue for each batch processing.
    """

    @li.work(assignment="context -> plan", form_param_key="branch", guidance=CODER_PROMPTS["plan_code"], capacity=2)
    async def _plan_code(self, branch=None, **kwargs):
        """
        Plans the code based on the given context. 
        It generates a step-by-step plan for the coding task.
        """
        if branch is None:
            branch = self.session.default_branch
        else:
            branch = self.session.branches[branch]
        print(f"*****\nPlanning code for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        result = await branch.chat(**kwargs)
        print(f"*****\nCode plan generated for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        return result
    
    @li.work(assignment="context, plan -> code", form_param_key="branch", guidance=CODER_PROMPTS["write_code"], capacity=2)
    async def _write_code(self, branch=None, **kwargs):
        """
        Writes the code according to the planned steps. 
        It uses the context and plan to generate the code.
        """
        if branch is None:
            branch = self.session.default_branch
        else:
            branch = self.session.branches[branch]
        print(f"*****\nWriting code for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        result = await branch.chat(**kwargs)
        print(f"*****\nCode written for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        return result
    
    @li.work(assignment="code -> review", form_param_key="branch", guidance=CODER_PROMPTS["review_code"], capacity=2)
    async def _review_code(self, branch=None, **kwargs):
        """
        Reviews the written code and provides feedback. 
        It ensures that the code meets the requirements and adheres to coding standards.
        """
        if branch is None:
            branch = self.session.default_branch
        else:
            branch = self.session.branches[branch]
        print(f"*****\nReviewing code for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        result = await branch.chat(**kwargs)
        print(f"*****\nCode reviewed for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        return result
    
    # handle function scheduling for execute: schedule next work
    async def handle_task(self, branch, work=None, refresh_time=1):
        """
        Manages the scheduling and execution of tasks. 
        It handles the transition between different steps (planning, writing, reviewing) based on the work status.
        """
        if work is None:
            task = await self._plan_code(branch=branch)
            if not self.work_functions["_plan_code"].worklog.queue.execution_mode:
                asyncio.create_task(self.work_functions["_plan_code"].worklog.queue.execute(refresh_time=refresh_time))
            return task
        if work.status == "COMPLETED":
            await self._update_form(work)
            if work.async_task_name == "_plan_code":
                task = await self._write_code(branch=branch)
                if not self.work_functions["_write_code"].worklog.queue.execution_mode:
                    asyncio.create_task(self.work_functions["_write_code"].worklog.queue.execute(refresh_time=refresh_time))
                return task
            elif work.async_task_name == "_write_code":
                task = await self._review_code(branch=branch)
                if not self.work_functions["_review_code"].worklog.queue.execution_mode:
                    asyncio.create_task(self.work_functions["_review_code"].worklog.queue.execute(refresh_time=refresh_time))
                return task
            elif work.async_task_name == "_review_code":
                return "COMPLETE"
        elif work.status == "FAILED":
            if branch is None:
                branch = self.session.default_branch
            failed_works = getattr(self, failed_works, [])
            failed_works.append((self.session.branches[branch], work))
            return "FAILED"
    
    # execute the whole process automatically and concurrently
    async def execute(self, refresh_time=1):
        """
        Executes the entire workflow automatically and concurrently. 
        It iterates through all branches and processes tasks in a loop until all tasks are completed.
        """
        for branch in self.session.branches:
            task = await self.handle_task(branch, refresh_time=refresh_time)
            if task is not None:
                self.tasks[branch.ln_id] = task
        
        while self.tasks:
            for work_func in self.work_functions.values():
                await work_func.worklog.forward()
        
            await asyncio.sleep(refresh_time)

            for branch_id in list(self.tasks.keys()):
                task = await self.handle_task(branch=branch_id, work=self.tasks[branch_id], refresh_time=refresh_time)
                if task == "COMPLETE" or task == "FAILED":
                    self.tasks.pop(branch_id)
                elif task is not None:
                    self.tasks[branch_id] = task
        
        await self.stop()

# Create Coder

In [6]:
coder = Coder(system=CODER_PROMPTS['system'])

### Create Coding Tasks

In [7]:
coder.create_new_issue("write a pure python function that takes a list of integers and returns the sum of all the integers in the list.")
coder.create_new_issue("write a pure python function that takes a list of integers and returns the maximum integer in the list.")
coder.create_new_issue("Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.")

# Execute Step-by-Step

## Plan code given context

### Schedule Function

With the `@li.work` decorator, calling the function will only parse it into a `work` and add it to the pending list of the internal `WorkLog`.
This decorator facilitates the scheduling and management of asynchronous tasks in the workflow.

In [8]:
# schedule the planning tasks for all branches in the Coder session.
for branch in coder.session.branches:
    await coder._plan_code(branch=branch.ln_id)
plan_log = coder.work_functions["_plan_code"].worklog

In [9]:
plan_log.pending

Progression(['ebad93cc5c3167b88cf9ca2fd8d3ba90', '5b3e50bd4ac2905ae0aff64d81e61961', 'f7c1bd6eeb002166776e0eabc507079b'])

In [10]:
plan_log.pile

                              ln_id                     created metadata  \
0  ebad93cc5c3167b88cf9ca2fd8d3ba90  2024-06-04T02:10:47.349831       {}   
1  5b3e50bd4ac2905ae0aff64d81e61961  2024-06-04T02:10:47.349939       {}   
2  f7c1bd6eeb002166776e0eabc507079b  2024-06-04T02:10:47.350019       {}   

  content embedding              status result error  \
0    None        []  WorkStatus.PENDING   None  None   
1    None        []  WorkStatus.PENDING   None  None   
2    None        []  WorkStatus.PENDING   None  None   

                                          async_task async_task_name  \
0  <coroutine object WorkFunction.perform at 0x14...      _plan_code   
1  <coroutine object WorkFunction.perform at 0x14...      _plan_code   
2  <coroutine object WorkFunction.perform at 0x14...      _plan_code   

  completion_timestamp duration lion_class  
0                 None     None       Work  
1                 None     None       Work  
2                 None     None       Work  

### Forward Work to WorkQueue

`worklog.forward()` will forward all pending `work` items to the internal `WorkQueue`, which manages the execution of these tasks.

In [11]:
await plan_log.forward() # forward to WorkQueue and wait for processing

### Process WorkQueue

`workqueue.process()` will execute the functions in the queue in batches. The batch size is determined by the capacity defined in `@li.work` decorator.

In [12]:
print("first batch:")
await plan_log.queue.process() # process first two functions in queue (as defined capacity=2)

first batch:
*****
Planning code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Planning code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code plan generated for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Code plan generated for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**


In [13]:
print("secend batch:")
await plan_log.queue.process()

secend batch:
*****
Planning code for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.
----------
**
*****
Code plan generated for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.
----------
**


In [14]:
# update the step result
for work in plan_log.pile:
    await coder._update_form(work)

## Write code given context and plan

In [15]:
# schedule the writing tasks for all branches in the Coder session.
for branch in coder.session.branches:
    await coder._write_code(branch=branch)
write_log = coder.work_functions["_write_code"].worklog

In [16]:
await write_log.forward() # forward to WorkQueue and wait for processing
print("first batch:")
await write_log.queue.process() # process first two functions in queue (as defined capacity=2)
print("secend batch:")
await write_log.queue.process()

first batch:
*****
Writing code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Writing code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code written for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code written for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
secend batch:
*****
Writing code for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.
----------
**
*****
Code written for issue: 
----------
Write a

In [17]:
for work in write_log.pile:
    await coder._update_form(work)

## Review code given code

In [18]:
# schedule the reviewing tasks for all branches in the Coder session.
for branch in coder.session.branches:
    await coder._review_code(branch=branch)
review_log = coder.work_functions["_review_code"].worklog

In [19]:
await review_log.forward() # forward to WorkQueue and wait for processing
print("first batch:")
await review_log.queue.process() # process first two functions in queue (as defined capacity=2)
print("secend batch:")
await review_log.queue.process()

first batch:
*****
Reviewing code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Reviewing code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code reviewed for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code reviewed for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
secend batch:
*****
Reviewing code for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.
----------
**
*****
Code reviewed for issue: 
---------

In [20]:
for work in review_log.pile:
    await coder._update_form(work)

# Results

In [21]:
from IPython.display import Markdown, display

for k in coder.forms.keys():
    display(Markdown("Context: " + coder.forms[k].context))
    display(Markdown("Plan: " + coder.forms[k].plan))
    display(Markdown("```python \n" + coder.forms[k].code + "\n```"))
    display(Markdown(coder.forms[k].review))
    print("*************************************************************************************************")

Context: write a pure python function that takes a list of integers and returns the sum of all the integers in the list.

Plan: {"step": "Define the function signature", "details": "Create a function named `sum_of_integers` that takes a single parameter `numbers`, which is expected to be a list of integers."}, {"step": "Initialize a variable to store the sum", "details": "Inside the function, initialize a variable `total_sum` to 0. This variable will be used to accumulate the sum of the integers in the list."}, {"step": "Iterate through the list", "details": "Use a for loop to iterate through each integer in the `numbers` list."}, {"step": "Add each integer to the sum", "details": "Within the loop, add the current integer to `total_sum`."}, {"step": "Return the total sum", "details": "After the loop completes, return the value of `total_sum`."}

```python 
def sum_of_integers(numbers):
    """
    Calculate the sum of all integers in a list.

    Parameters:
    numbers (list of int): A list of integers to be summed.

    Returns:
    int: The sum of all integers in the list.
    """
    total_sum = 0

    for number in numbers:
        total_sum += number

    return total_sum

```

The provided code is well-written and follows the given plan accurately. It includes a clear docstring that describes the function's purpose, parameters, and return value. The code is concise and adheres to Python's best practices. No unnecessary markdown or descriptions are present. The function is ready for use as is.

*************************************************************************************************


...

Context: Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.

Plan: 1. Define a function named 'count_integers' that takes a single parameter 'integer_list', which is expected to be a list of integers., 2. Initialize an empty dictionary named 'counts' to store the counts of each integer., 3. Iterate over each integer in 'integer_list'., 4. For each integer, check if it is already a key in the 'counts' dictionary., 5. If the integer is already a key, increment its value by 1., 6. If the integer is not a key, add it to the 'counts' dictionary with a value of 1., 7. After the loop completes, return the 'counts' dictionary.

```python 
def count_integers(integer_list):
    """
    Count the occurrences of each integer in the given list.

    Parameters:
    integer_list (list): A list of integers.

    Returns:
    dict: A dictionary where keys are integers from the list and values are the counts of how many times each integer appears in the list.
    """
    counts = {}

    for integer in integer_list:
        if integer in counts:
            counts[integer] += 1
        else:
            counts[integer] = 1

    return counts
```

The provided code is well-structured and follows the given plan accurately. The docstring is clear and provides useful information about the function's purpose, parameters, and return value. There are no unnecessary markdown or descriptions in the code. The function is concise and adheres to the coding standards. No changes are necessary.

*************************************************************************************************


# Execute Automatically and Concurrently

In [22]:
coder_auto = Coder()

In [23]:
coder_auto.create_new_issue("write a pure python function that takes a list of integers and returns the sum of all the integers in the list.")
coder_auto.create_new_issue("write a pure python function that takes a list of integers and returns the maximum integer in the list.")
coder_auto.create_new_issue("Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.")

In [24]:
await coder_auto.execute(refresh_time=1)

*****
Planning code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Planning code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code plan generated for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Writing code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code plan generated for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Planning code for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the 

In [25]:
from IPython.display import Markdown, display

for k in coder.forms.keys():
    display(Markdown("Context: " + coder.forms[k].context))
    display(Markdown("Plan: " + coder.forms[k].plan))
    display(Markdown("```python \n" + coder.forms[k].code + "\n```"))
    display(Markdown(coder.forms[k].review))
    print("*************************************************************************************************")

Context: write a pure python function that takes a list of integers and returns the sum of all the integers in the list.

Plan: {"step": "Define the function signature", "details": "Create a function named `sum_of_integers` that takes a single parameter `numbers`, which is expected to be a list of integers."}, {"step": "Initialize a variable to store the sum", "details": "Inside the function, initialize a variable `total_sum` to 0. This variable will be used to accumulate the sum of the integers in the list."}, {"step": "Iterate through the list", "details": "Use a for loop to iterate through each integer in the `numbers` list."}, {"step": "Add each integer to the sum", "details": "Within the loop, add the current integer to `total_sum`."}, {"step": "Return the total sum", "details": "After the loop completes, return the value of `total_sum`."}

```python 
def sum_of_integers(numbers):
    """
    Calculate the sum of all integers in a list.

    Parameters:
    numbers (list of int): A list of integers to be summed.

    Returns:
    int: The sum of all integers in the list.
    """
    total_sum = 0

    for number in numbers:
        total_sum += number

    return total_sum

```

The provided code is well-written and follows the given plan accurately. It includes a clear docstring that describes the function's purpose, parameters, and return value. The code is concise and adheres to Python's best practices. No unnecessary markdown or descriptions are present. The function is ready for use as is.

*************************************************************************************************


...

Context: Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.

Plan: 1. Define a function named 'count_integers' that takes a single parameter 'integer_list', which is expected to be a list of integers., 2. Initialize an empty dictionary named 'counts' to store the counts of each integer., 3. Iterate over each integer in 'integer_list'., 4. For each integer, check if it is already a key in the 'counts' dictionary., 5. If the integer is already a key, increment its value by 1., 6. If the integer is not a key, add it to the 'counts' dictionary with a value of 1., 7. After the loop completes, return the 'counts' dictionary.

```python 
def count_integers(integer_list):
    """
    Count the occurrences of each integer in the given list.

    Parameters:
    integer_list (list): A list of integers.

    Returns:
    dict: A dictionary where keys are integers from the list and values are the counts of how many times each integer appears in the list.
    """
    counts = {}

    for integer in integer_list:
        if integer in counts:
            counts[integer] += 1
        else:
            counts[integer] = 1

    return counts
```

The provided code is well-structured and follows the given plan accurately. The docstring is clear and provides useful information about the function's purpose, parameters, and return value. There are no unnecessary markdown or descriptions in the code. The function is concise and adheres to the coding standards. No changes are necessary.

*************************************************************************************************
